# merging DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# general configs
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Merging DM_RETAIL_CLIE & BTT_ASEGURADOS

## Loading DM_RETAIL_CLIE

In [3]:
# path to dataset
path_retail = "../../../data/interim/3. row_red/DM_RETAIL_CLIE-red_col-red_cols-clean-unique_CIF_ID.feather"

if not(os.path.exists(path_retail) and os.path.isfile(path_retail)):
    raise Exception("File {} doesn't exists.".format(path_retail))

In [4]:
df_retail = pd.read_feather(path_retail)
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741910 entries, 0 to 741909
Data columns (total 31 columns):
ACA                    741910 non-null bool
ANTIG_MESES            741466 non-null float64
APTO_E_MAIL            741910 non-null bool
CANAL                  738563 non-null object
CANAL_ORIGEN           741910 non-null object
CIF_ID                 741909 non-null float64
COBERTURA_PLAN         715940 non-null object
COD_MARCA              715940 non-null float64
COD_MOD                715940 non-null float64
DESC_EQUIPO            161921 non-null object
ES_ELEGIBLE            741910 non-null object
FLAG_EBILLING          741910 non-null bool
FLAG_ROBO_CONTENIDO    741910 non-null bool
FOR_COBRO              741910 non-null object
GRUPO_COMBUSTIBLE      715684 non-null float64
MARCA                  715940 non-null object
MARCA_SIMPLIFICADA     715940 non-null object
MCA_0KM                732395 non-null float64
MODELO                 715940 non-null object
NEGOCIO          

<b>DM_RETAIL_CLIE: has both keys CIF_ID & NUM_SECU_POL</b>

In [7]:
# checking ID is unique
df_retail["NUM_SECU_POL"].nunique() / len(df_retail)

1.0

In [6]:
# checking ID is unique
df_retail["CIF_ID"].nunique() / len(df_retail)

0.9999986521276165

## Loading BTT_ASEGURADOS

In [14]:
# path to dataset
path_aseg = "../../../data/interim/3. row_red/BTT_ASEGURADOS-red_col-red_cols-clean-unique_CIF_ID.feather"

if not(os.path.exists(path_aseg) and os.path.isfile(path_aseg)):
    raise Exception("File {} doesn't exists.".format(path_aseg))

In [15]:
df_aseg = pd.read_feather(path_aseg)
df_aseg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1271360 entries, 0 to 1271359
Data columns (total 4 columns):
CIF_ID            1271360 non-null float64
COD_EST_CIVIL     1210521 non-null object
FECHA_DESDE       1248383 non-null datetime64[ns]
TIPO_ACTIVIDAD    1271360 non-null object
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 38.8+ MB


In [16]:
# checking ID is unique
df_aseg["CIF_ID"].nunique() / len(df_aseg)

1.0

## Comparing DM_RETAIL_CLIE vs BTT_ASEGURADOS
Checking CIF_ID intersection

In [17]:
# Checking intersection between both datasets
id_common = set(df_retail["CIF_ID"].unique()).intersection(set(df_aseg["CIF_ID"].unique()))
len(id_common), len(id_common) / len(df_retail) *100, len(id_common) / len(df_aseg)*100

(478964, 64.55823482632664, 37.67335766423358)

<b>Too much missing rows in both datasets</b>

## Merging DM_RETAIL_CLIE & BTT_ASEGURADOS

In [18]:
# Merging dropping not founded INVESTIGACIONES
df_ret_aseg = pd.merge(df_retail, df_aseg, on="CIF_ID", how="left", suffixes=("_retail", "_aseg"))
print(len(df_retail), len(df_aseg), len(df_ret_aseg))

741910 1271360 741910


In [20]:
df_ret_aseg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 741910 entries, 0 to 741909
Data columns (total 34 columns):
ACA                    741910 non-null bool
ANTIG_MESES            741466 non-null float64
APTO_E_MAIL            741910 non-null bool
CANAL                  738563 non-null object
CANAL_ORIGEN           741910 non-null object
CIF_ID                 741909 non-null float64
COBERTURA_PLAN         715940 non-null object
COD_MARCA              715940 non-null float64
COD_MOD                715940 non-null float64
DESC_EQUIPO            161921 non-null object
ES_ELEGIBLE            741910 non-null object
FLAG_EBILLING          741910 non-null bool
FLAG_ROBO_CONTENIDO    741910 non-null bool
FOR_COBRO              741910 non-null object
GRUPO_COMBUSTIBLE      715684 non-null float64
MARCA                  715940 non-null object
MARCA_SIMPLIFICADA     715940 non-null object
MCA_0KM                732395 non-null float64
MODELO                 715940 non-null object
NEGOCIO          

### temporal saving (checkpoint)

In [21]:
path_to_save = "../../../data/interim/5. merged"
check_filename = "DM_RETAIL+BTT_ASEGURADOS.feather"
df_ret_aseg.to_feather(os.path.join(path_to_save, check_filename))

# MERGIN DM_RETAIL+BTT_ASEGURADOS & VIGABT_POLIZAS

## Restart from checkpoint (DM_RETAIL+BTT_ASEGURADOS)

In [3]:
path = "../../../data/interim/5. merged"
check_filename = "DM_RETAIL+BTT_ASEGURADOS.feather"
path_file = os.path.join(path, check_filename)
df_ret_aseg = pd.read_feather(path_file)
df_ret_aseg.info()

/home/voolkia/anaconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741910 entries, 0 to 741909
Data columns (total 34 columns):
ACA                    741910 non-null bool
ANTIG_MESES            741466 non-null float64
APTO_E_MAIL            741910 non-null bool
CANAL                  738563 non-null object
CANAL_ORIGEN           741910 non-null object
CIF_ID                 741909 non-null float64
COBERTURA_PLAN         715940 non-null object
COD_MARCA              715940 non-null float64
COD_MOD                715940 non-null float64
DESC_EQUIPO            161921 non-null object
ES_ELEGIBLE            741910 non-null object
FLAG_EBILLING          741910 non-null bool
FLAG_ROBO_CONTENIDO    741910 non-null bool
FOR_COBRO              741910 non-null object
GRUPO_COMBUSTIBLE      715684 non-null float64
MARCA                  715940 non-null object
MARCA_SIMPLIFICADA     715940 non-null object
MCA_0KM                732395 non-null float64
MODELO                 715940 non-null object
NEGOCIO          

## Loading VIGABT_POLIZAS

In [4]:
# path to dataset
path_vigabt = "../../../data/interim/3. row_red/VIGABT_POLIZAS-red_col-red_rows-clean.feather"

if not(os.path.exists(path_vigabt) and os.path.isfile(path_vigabt)):
    raise Exception("File {} doesn't exists.".format(path_vigabt))

In [5]:
df_vigabt = pd.read_feather(path_vigabt)
df_vigabt.info()

/home/voolkia/anaconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729756 entries, 0 to 729755
Data columns (total 29 columns):
ANTIG                           708640 non-null float64
CANT_RENOVACION                 709911 non-null float64
CAPITAL_ACCESORIOS              729756 non-null object
CAPITAL_ASEGURADO_COTIZACION    725415 non-null object
CAPITAL_VEHICULO                729756 non-null object
COD_COBRO                       729756 non-null object
COD_COBRO_ANTERIOR              604173 non-null object
COD_POSTAL                      729756 non-null int64
COD_PROD                        729756 non-null int64
COD_RAMO                        729756 non-null int64
COD_RIES                        729756 non-null int64
COD_ZONA_CASCO                  729754 non-null float64
COD_ZONA_RC                     729754 non-null float64
COD_ZONA_ROBO                   729754 non-null float64
CONV_COMISIONARIO               729756 non-null object
FECHA_PROCESO                   729756 non-null datetime64[ns]


In [6]:
# checking unique key
df_vigabt["NUM_SECU_POL"].nunique() / len(df_vigabt)

1.0

In [7]:
# Dropping dates
df_vigabt.drop(columns=["FECHA_PROCESO", "FECHA_VENC_POL", "FECHA_VIG_ORIG_POL", "FECHA_VIG_POL"], inplace=True)
df_vigabt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729756 entries, 0 to 729755
Data columns (total 25 columns):
ANTIG                           708640 non-null float64
CANT_RENOVACION                 709911 non-null float64
CAPITAL_ACCESORIOS              729756 non-null object
CAPITAL_ASEGURADO_COTIZACION    725415 non-null object
CAPITAL_VEHICULO                729756 non-null object
COD_COBRO                       729756 non-null object
COD_COBRO_ANTERIOR              604173 non-null object
COD_POSTAL                      729756 non-null int64
COD_PROD                        729756 non-null int64
COD_RAMO                        729756 non-null int64
COD_RIES                        729756 non-null int64
COD_ZONA_CASCO                  729754 non-null float64
COD_ZONA_RC                     729754 non-null float64
COD_ZONA_ROBO                   729754 non-null float64
CONV_COMISIONARIO               729756 non-null object
MCA_AGRAVANTE                   729582 non-null object
MCA_EMPL

## Comparing merged vs VIGABT_POLIZAS
Checking NUM_SECU_POL intersection

In [8]:
# Checking intersection between both datasets
id_common = set(df_ret_aseg["NUM_SECU_POL"].unique()).intersection(set(df_vigabt["NUM_SECU_POL"].unique()))
len(id_common), len(id_common) / len(df_ret_aseg) *100, len(id_common) / len(df_vigabt)*100

(7164, 0.9656157754983759, 0.9816979922056139)

Only an 1% of matching. <br>
<b>Decision: DO NOT merge VIGABT_POLIZAS</b>

## Merging merged & VIGABT_POLIZAS

<h3 style="color:red">DO NOT merge it</h3>

In [13]:
# 


### Saving merged file

# Merging (DM_RETAIL_CLIE & BTT_ASEGURADOS) & tb_cif

## Restart from checkpoint (DM_RETAIL+BTT_ASEGURADOS)

In [3]:
path = "../../../data/interim/5. merged"
check_filename = "DM_RETAIL+BTT_ASEGURADOS.feather"
path_file = os.path.join(path, check_filename)
df_ret_aseg = pd.read_feather(path_file)
df_ret_aseg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741910 entries, 0 to 741909
Data columns (total 34 columns):
ACA                    741910 non-null bool
ANTIG_MESES            741466 non-null float64
APTO_E_MAIL            741910 non-null bool
CANAL                  738563 non-null object
CANAL_ORIGEN           741910 non-null object
CIF_ID                 741909 non-null float64
COBERTURA_PLAN         715940 non-null object
COD_MARCA              715940 non-null float64
COD_MOD                715940 non-null float64
DESC_EQUIPO            161921 non-null object
ES_ELEGIBLE            741910 non-null object
FLAG_EBILLING          741910 non-null bool
FLAG_ROBO_CONTENIDO    741910 non-null bool
FOR_COBRO              741910 non-null object
GRUPO_COMBUSTIBLE      715684 non-null float64
MARCA                  715940 non-null object
MARCA_SIMPLIFICADA     715940 non-null object
MCA_0KM                732395 non-null float64
MODELO                 715940 non-null object
NEGOCIO          

## Loading tb_cif

In [4]:
# path to dataset
path_cif = "../../../data/interim/2. col_red_min/tb_cif-red_col-red_rows-clean.feather"

if not(os.path.exists(path_cif) and os.path.isfile(path_cif)):
    raise Exception("File {} doesn't exists.".format(path_cif))

In [6]:
df_cif = pd.read_feather(path_cif)
df_cif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174922 entries, 0 to 174921
Data columns (total 6 columns):
CLIENTE                    129422 non-null object
CODIGO_NACION              172133 non-null object
DATECO_TIPO_ACTIVIDAD      112412 non-null object
DOMICILIO_CODIGO_POSTAL    148152 non-null float64
HABILITADO                 38716 non-null object
CIF_ID                     174922 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 8.0+ MB


In [7]:
# checking unique key
df_cif["CIF_ID"].nunique() / len(df_cif)

1.0

## Comparing merged vs tb_cif
Checking CIF_ID intersection

In [8]:
# Checking intersection between both datasets
id_common = set(df_ret_aseg["CIF_ID"].unique()).intersection(set(df_cif["CIF_ID"].unique()))
len(id_common), len(id_common) / len(df_ret_aseg) *100, len(id_common) / len(df_cif)*100

(111794, 15.06840452345972, 63.91077165822481)

<b>Few rows in common</b>

## Merging merged & tb_cif


In [10]:
# Merging dropping not founded INVESTIGACIONES
df_ret_aseg_cif = pd.merge(df_ret_aseg, df_cif, on="CIF_ID", how="left", suffixes=("", "_tbcif"))
print(len(df_ret_aseg), len(df_cif), len(df_ret_aseg_cif))

741910 174922 741910


### saving merged dataset

In [11]:
path_to_save = "../../../data/interim/5. merged"
check_filename = "DM_RETAIL+BTT_ASEGURADOS+tb_cif.feather"
df_ret_aseg_cif.to_feather(os.path.join(path_to_save, check_filename))